`wu_generating_data_integer_taus_zero_phase_unwrapped.ipynb`

> Generating data for the wrap-unwrap problem

> integer values of tau, zero phase or noise (i want to make this easy)

> UNWRAPPED DATA

##### imports & notebook setup

In [5]:
import numpy as np, pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, FuncFormatter

from plot_styles import set_plotstyles
set_plotstyles()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'

In [6]:
# frequency channels for all data

min_freq = 0.1   # GHz
max_freq = 0.2   # GHz
num_freqs = 1024
freqs = np.linspace(min_freq, max_freq, num_freqs) # GHz

In [7]:
# tau bounds
min_tau = -100 # ns
max_tau =  100 # ns

In [8]:
def get_slope(y, x = freqs):
    """
    Computes the mean slope of the data.
    Assumes the data is a *perfect* line
    
    Parameters
    ---------
    y (array) : complex data to be unwrapped
    x (array) : frequencies [GHz]
    
    Returns
    -------
    (float)  : slope of the line [nano-seconds]
    
    """
    return np.mean((np.diff(np.unwrap(y))  / (2 * np.pi)) / np.diff(x))

#### Generate data

> Data should have:
 - a low rate of spikes
 - a small amount of noise
  - occasional large amounts of noise
 - integer values of tau

In [13]:
# how many entries in the dataframe?
num_entries = 10000

# each entry has a slope computed before noise & spikes are applied
tau_col = np.empty(num_entries)

# generate data
data = np.empty((num_entries, num_freqs))
for i in range(num_entries):
    
    # fix tau
    tau = np.random.randint(min_tau,high = max_tau) 
    
    # fix phi
    phi = 0
    
    # generate angle data from complex data
    data[i]    = np.angle(np.exp(2*np.pi*1j * (tau*freqs + phi)))
    
    # compute slope
    tau_col[i] = get_slope(data[i])

    data[i] = np.unwrap(data[i])

#### save data

> save as a compressed file

> Filename convention:

`wu_10000_normal-uniform-noise_5-percent-normal-pi-spike_integer-taus.bz2`

"wrap-unwrap" \_ "number of entries" \_ "type of noise added" \_ "type of spikes added" \_ "comments"

In [14]:
df = pd.DataFrame(data)
df.insert(0, 'tau', tau_col)
df.to_csv('data/wu_10000_zero-noise_zero-spikes_integer-taus-unwrapped', index = False, compression = 'bz2')